Setup umbrella sampling approach
---

We use [umbrella sampling](http://www.mdtutorials.com/gmx/umbrella/index.html) for the calculations of the Potential of Mean Force (PMF) of adsorption of PLGA polymer on the gold nanoparticle.

## Libraries

In [1]:
import os, sys
import shutil

## Create the topology <a name="S1"></a> 

We use [`Packmol`](http://leandro.iqm.unicamp.br/m3g/packmol/userguide.shtml) code:

1. To rotate the gold nanoparticle so as to calculate the PMF between different angular positions of the AuNP and the PLGA;
2. To place the PLGA as close as possible to the gold nanoparticle surface.

### Rotate the AuNP <a name="S1.1"></a> 

> **Note**: We rotate the AuNP around the axis r, orthogonal to both crystalline plane normal vectors (see the schematic NP sketch in figure below).

> The axis and rotation angles are calculated in the jupyter notebook: `4.1-rotation_NP.ipynb` 

<p align="left">
<img src="../img/np_sketch.png" width="30%" height ="30%" style="background-color:white">
</p>

> $\theta = 0.0°$ and $\theta = 54.7°$ have the reaction coordinate $\xi$ perpendicular to {1 0 0} and {1 1 1} planes, respectively, while the other two orientation show intermediate cases.

Change the variables `theta` and `fixed` in the following way:

1. `theta` = 0 , `fixed 35.0 35.0 35.0 0.0 0.0 0.0` 
2. `theta` = 18 , `fixed 35.0 35.0 35.0 0.229013 -0.223233 0.025779`
3. `theta` = 36 , `fixed 35.0 35.0 35.0 0.481915 -0.434005 0.108249`
4. `theta` = 54 , `fixed 35.0 35.0 35.0 0.785398 -0.615479 0.261799`

> Remember to change the name of the `output` file, to not overwrite them.

In [24]:
%cd ../simulations/0-Packmol/
file_NP = 'AuNP_ASE_wulff_surf_d39.pdb'       
theta = 54

with open(os.path.join('inputs_US', 'packmol_%d.in' %theta), 'w' ) as file_obj:
    file_obj.write(
        """
filetype pdb
tolerance 2.0  

structure {:s}
number 1
center
fixed 35.0 35.0 35.0 0.785398 -0.615479 0.261799
end structure 

output AuNP_54.7.pdb         

        """.format(file_NP, theta)
    )

[Errno 2] No such file or directory: '../simulations/0-Packmol/'
/home/roberta/SM_AuNP_PLGA/simulations/0-Packmol


In [ ]:
# Change the name of the packmol input 

%cd inputs_US/
!packmol < packmol_54.in 

### Place the PLGA <a name="S1.2"></a> 

Change the variables `theta` and `file_NP` in the following way:

1. `theta` = 0 , `file_NP` AuNP_0.0.pdb
2. `theta` = 18 , `file_NP` AuNP_18.2.pdb
3. `theta` = 36 , `file_NP` AuNP_36.5.pdb
4. `theta` = 54 , `file_NP` AuNP_54.7.pdb

> Remember to change the name of the `output` file, to not overwrite them.

In [38]:
%cd ..

/home/roberta/SM_AuNP_PLGA/simulations/0-Packmol


In [39]:
file_NP = 'AuNP_54.7.pdb' 
file_PLGA = '2PLGA_chain_conect.pdb'      
theta = 54

with open(os.path.join('inputs_US', 'packmol_tot_%d.in' %theta), 'w' ) as file_obj:
    file_obj.write(
        """
filetype pdb
tolerance 2.0  

structure {:s}
number 1
center
fixed 35.0 35.0 35.0 0.0 0.0 0.0
end structure 

structure {:s}
  number 1
  center
  fixed 35.0 35.0 57.0 0.0 0.0 0.0
end structure 

output ic_54.7.pdb         

        """.format(file_NP, file_PLGA, theta)
    )

In [ ]:
# Change the name of the packmol input 

%cd inputs_US/
!packmol < packmol_tot_54.in 

In [49]:
!pwd

/home/roberta/SM_AuNP_PLGA/simulations/0-Packmol/inputs_US


In [57]:
#We copy the input files into their respective folders changing the variable `theta_`
angle= 54.7

shutil.copyfile(os.path.join(f"ic_{angle}.pdb"), 
                os.path.join("..","..", "2-US_free_energy", f"theta_{angle}", "TOP","ic.pdb"))

'../../2-US_free_energy/theta_54.7/TOP/ic.pdb'

In [59]:
%cd ../../2-US_free_energy/

/home/roberta/SM_AuNP_PLGA/simulations/2-US_free_energy


Then, we generate the topology through `GROMACS` software, using the mixed force field `mix_oplsaa.ff`.

In [ ]:
# Change the folder name and generate the topology for all the configurations

%cd theta_54.7//TOP
!gmx pdb2gmx -f ic.pdb -o ic.gro -water spce -ff mix_oplsaa

> Remember to change the name of the position restrains in POSRES_GOLD and in POSRES_PLGA for the files `topol_Other.itp` and `topol_Other_chain_A.itp`, respectively

> **NOTE**: We describe the US protocol followed for a single configuration ($\theta=54.7°$) because it is the same also for the others.

## Our US protocol <a name="S2"></a> 

### Define box and carry out solvation <a name="S2.1"></a> 

In [ ]:
%cd ../BOX
!gmx editconf -f ../TOP/ic.gro -o newbox.gro -c no -center 3.5 3.5 3.5 -box 7.0 7.0 13.0 -bt triclinic
!gmx solvate -cp newbox.gro -cs spc216.gro -o solv.gro -p ../TOP/topol.top

### Energy minimization <a name="S2.2"></a> 

In [ ]:
%cd ../EM
!gmx grompp -f minim.mdp -c ../BOX/solv.gro -p ../TOP/topol.top -o em.tpr  
!gmx mdrun -v -deffnm em	

### Equilibration <a name="S2.3"></a> 

We apply the canonical ensemble `NVT` to reach an equilibrium temperature of 300 K

In [ ]:
%cd ../NVT
!gmx grompp -f nvt.mdp -r ../EM/em.gro -c ../EM/em.gro -p ../TOP/topol.top -o nvt.tpr  
!gmx mdrun -v -deffnm nvt					

We set the isothermal-isobaric ensemble `NPT` at an equilibrium pressure of 1 bar

In [ ]:
%cd ../NPT
!gmx grompp -f npt.mdp -r ../NVT/nvt.gro -c ../NVT/nvt.gro -p ../TOP/topol.top -o npt.tpr  
!gmx mdrun -v -deffnm npt	

### Generate configurations <a name="S2.4"></a> 

We carry out 2.5 ns MD simulation in which the NP is constrained to its initial position while the PLGA is pulled along the z-axis.

> **NOTE**: We need to define some custom index groups (`gold` and `plga`) for this pulling simulation, using [make_ndx](https://manual.gromacs.org/current/onlinehelp/gmx-make_ndx.html).

In [ ]:
%cd ../PULL
!gmx grompp -f md_pull.mdp -r ../NPT/npt.gro -c ../NPT/npt.gro -t ../NPT/npt.cpt -n ../NPT/index.ndx -p ../TOP/topol.top -o pull.tpr  
!gmx mdrun -v -deffnm pull -pf pullf.xvg -px pullx.xvg


Saving snapshots every 1 ps, the reaction coordinate is divided into single sampling windows.

> `echo 0` means to execute the command for the whole system (AuNP, PLGA and water) 

In [ ]:
%cd ..
!mkdir CONF
%cd CONF

!echo 0|gmx trjconv -s ../PULL/pull.tpr -f ../PULL/pull.xtc -o conf.gro -sep 

### Umbrella sampling <a name="S2.5"></a> 

For each selected window we re-run our MD protocol starting from equilibration steps and concluding with a 10 ns of production run.

> Here, I show the command lines for the first windows (`conf0.gro`). The input `conf*.gro` must be changed according to the window you choose to consider. 

In [ ]:
%cd ../UMB

!gmx grompp -f nvt_umbrella.mdp -r ../CONF/conf0.gro -c ../CONF/conf0.gro -p ../TOP/topol.top -n ../NPT/index.ndx -o nvt0.tpr  
!gmx mdrun -v -deffnm nvt0

!gmx grompp -f npt_umbrella.mdp -r ../NVT/nvt0.gro -c ../NVT/nvt0.gro -p ../TOP/topol.top -n ../NPT/index.ndx -o npt0.tpr  
!gmx mdrun -v -deffnm npt0

!gmx grompp -f md_umbrella.mdp -r ../NPT/npt0.gro -c ../NPT/npt0.gro -t ../NPT/npt0.cpt -p ../TOP/topol.top -n ../NPT/index.ndx -o umbrella0.tpr  
!gmx mdrun -v -deffnm umbrella0

### [Weighted histogram analysis method](http://www.alchemistry.org/wiki/Weighted_Histogram_Analysis_Method) <a name="S2.6"></a> 

We obtain the PMF profile using WHAM included in GROMACS.

The input to wham consists of two files, one that lists the names of the .tpr files of each window (`tpr-files.dat`), and the other that lists the names of either the pullf.xvg files from each window (`pullf-files.dat`).

In [ ]:
%cd ../
!mkdir ANALYSIS
%cd ANALYSIS
!gmx wham -it tpr-files.dat -if pullf-files.dat -o -hist -bsres -bsprof -unit kJ -nBootstrap 50 -bs-method b-hist 